In [ ]:
import os
import numpy as np
import struct
from PIL import Image
from timeit import default_timer as timer

outputPath = '../Resources/KittiDataset/data_road/testing/bev_velodyne/'
example = None

minX = 6
maxX = 46
minY = -10
maxY = 10

def read_bin_velodyne(path):
    pc_list=[]
    with open(path,'rb') as f:
        content=f.read()
        pc_iter=struct.iter_unpack('ffff',content)
        for idx,point in enumerate(pc_iter):
            pc_list.append([point[0],point[1],point[2], point[3]])
    return np.asarray(pc_list,dtype=np.float32)

def main():
    global example
    root_dir='../Resources/KittiDataset/data_road/testing/velodyne/'
    filename=os.listdir(root_dir)
    file_number=len(filename)

    #print(file_number)

    for i in range(file_number):
        #print(i)
        path=os.path.join(root_dir, filename[i])
        #print(path)
        #print(filename[i])
        outputFilename = filename[i].split('.')[0] + '.png'
        pcd=read_bin_velodyne(path)

        grid = np.zeros((800,400))
        grid2 = np.zeros((800,400))
        grid3 = np.zeros((800,400))

        for p in pcd:
            x = p[0]
            y = p[1]
            z = p[2]
            i = p[3]

            if minX<x<maxX and minY<y<maxY:
                #In bounds
                X = int((x-6)/0.05)-1
                Y = int((y+10)/0.05)-1
                grid[X][Y] = 255
                grid2[X][Y] = i
                grid3[X][Y] = z

        grid = np.rot90(grid, 2)
        grid2 = np.rot90(grid2, 2)
        grid3 = np.rot90(grid3, 2)

        #Normalize i channel
        grid2 *= 255

        # Normalize z channel
        grid3 = grid3.clip(min=-1.8, max=-1.2)
        grid3 = (255*(grid3 - np.min(grid3))/np.ptp(grid3)).astype(float)
        grid3 = 255-grid3

        im1 = Image.fromarray(grid)
        im2 = Image.fromarray(grid2)
        im3 = Image.fromarray(grid3)

        im1 = im1.convert("L")
        im2 = im2.convert("L")
        im3 = im3.convert("L")

        image = Image.merge('RGB', (im1,im2,im3))

        #im1.show()
        #im2.show()
        #im3.show()
        #image.show()
        image.save(outputPath+outputFilename)
if __name__=="__main__":
    start = timer()
    main()
    end = timer()

    print ("Elapsed time (seconds):" + str(end-start))

In [ ]:
print("Done")